In [86]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from scipy.spatial.distance import euclidean, cosine, chebyshev, braycurtis
import collections

In [78]:
class custom_knn(BaseEstimator):
    
    """
    
    My own implementation of KNN. The two parameters accepted are:
    1. int - the number of neighbors K
    2. function - the distance metric to use. The distance function must take in 2 numpy arrays and return a single distance. 
    
    The algorithm used will be the brute force approach. 
    
    Ideas taken from: 
    https://towardsdatascience.com/machine-learning-basics-with-the-k-nearest-neighbors-algorithm-6a6e71d01761
    https://github.com/scikit-learn-contrib/project-template/blob/master/skltemplate/_template.py
    
    """
    
    
    def __init__(self, n_neighbors=5, distance_func=euclidean):
        self.n_neighbors = n_neighbors
        self.distance_func = distance_func
        
    def fit(self, X, Y):
        
        """
        X and y must be numpy arrays
        """
        
        # Store training set as model objects
        self.X_ = X
        self.Y_ = Y
        
        # That's it for training. Since knn is a lazy algorithm, the only training it needs is to memorize the training data and its labels. 
        return(self)
    
    
    
    def predict(self, X_test, **kwargs):
        
        """
        Parameters:
        X_test - the evaluation set. Each instance in X_test will have distances calculated against each point in X_train
        optional kwargs to pass to the distance function. 
        """
        
        # Check that .fit has been called
        # Caveat: X and Y must share a 1:1 mapping of indices
        check_is_fitted(self, ['X_', 'Y_'])
        
        
        # Initialize dictionary to store results
        results = {}
        
        for i_test, inst in enumerate(X_test):
            
            # We need to find the k nearest neighbors for each instance of the training set to the query point inst
            # Initialize temp dictionary to store results. Keyed by index of training set, and contains distance
            
            inst_dict = {}
            
            for i_train, query in enumerate(self.X_):
                
                # Calculate the distance
                inst_dict[i_train] = self.distance_func(inst, query, **kwargs)
                
            # Extract the training indices for the K nearest neighbors by our specified distance metric
            n_nearest = [k for k,v in sorted(inst_dict.items(), key=lambda x: x[1])[:self.n_neighbors]]
            
            # Take a vote!
            y_labels = self.Y_[n_nearest]
            
            # Count frequences, select the highest count, and extract the value from the resultant tuple
            # Note: If there is a tie, the first element is selected. This behavior is a little random by nature. 
            # Sklearn's KNN does the same thing, but the order of the elements may be different. 
            vote = collections.Counter(y_labels).most_common(1)[0][0]
            
            # Add the vote to our results dictionary
            results[i_test] = vote
            
        # After looping, return an array of predictions
        return np.array(list(results.values()))
    
    

In [75]:
from sklearn import datasets
wine = datasets.load_wine()

In [76]:
wine.data.shape

(178, 13)

Let's test out our classifier, and see how close our results are to sklearn's version of KNN. 

In [85]:
from sklearn.neighbors import KNeighborsClassifier

our_knn = custom_knn(n_neighbors=5, distance_func=euclidean)
sklearn_knn = KNeighborsClassifier(n_neighbors=5, algorithm='brute', metric='euclidean')

our_knn.fit(wine.data, wine.target)
sklearn_knn.fit(wine.data, wine.target)

sum(our_knn.predict(wine.data) == sklearn_knn.predict(wine.data)) / wine.data.shape[0]

0.9550561797752809

We get the same prediction results as sklearn's KNN on 95.5% of all entries in our dataset. 

We can use our custom classifier to do some actual data mining. As we would do with a built-in KNN module, we can perform some grid search to tune our hyperparameters, using simple accuracy as a scoring metric. 

In [94]:
%%time
# Idea taken from https://nbviewer.jupyter.org/github/jakemdrew/StrandPy/blob/master/StrandClassifierV2%20GridSearchCV%20RDP%20CategoryScores.ipynb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# create cross validation iterator
cv = StratifiedKFold(n_splits=10, random_state=8)
knn_slate = custom_knn()

parameters = { 'n_neighbors':[3,5,7,9,11,13,15]
              ,'distance_func': [euclidean, cosine, chebyshev, braycurtis]
             }

grid_search = GridSearchCV(estimator=knn_slate
                   , n_jobs=-1 # jobs to run in parallel
                   , verbose=1 # low verbosity in output messages
                   , param_grid=parameters
                   , cv=cv
                   , scoring='accuracy')

#Perform hyperparameter search to find the best combination of parameters for our data
grid_search.fit(wine.data, wine.target)

Fitting 10 folds for each of 28 candidates, totalling 280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    4.0s


CPU times: user 981 ms, sys: 39.4 ms, total: 1.02 s
Wall time: 12.7 s


[Parallel(n_jobs=-1)]: Done 280 out of 280 | elapsed:   12.7s finished
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=8, shuffle=False),
             error_score='raise-deprecating',
             estimator=custom_knn(distance_func=<function euclidean at 0xa1c414b90>,
                                  n_neighbors=5),
             iid='warn', n_jobs=-1,
             param_grid={'distance_func': [<function euclidean at 0xa1c414b90>,
                                           <function cosine at 0xa1c414d40>,
                                           <function chebyshev at 0xa1c415170>,
                                           <function braycurtis at 0xa1c415200>],
                         'n_neighbors': [3, 5, 7, 9, 11, 13, 15]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=1)

From the output, we see that overall GridSearch took about 12.7 seconds in total. In detail, the timing for each step is given as:

In [101]:
def mean(x): # helper function for calculating array  means
    return(sum(x) / len(x))

print("Mean fit time is {}s".format(mean(grid_search.cv_results_['mean_fit_time'])))
print("Mean score time is {}s".format(mean(grid_search.cv_results_['mean_score_time'])))

Mean fit time is 6.043570382254463e-05s
Mean score time is 0.17863712395940515s


In [90]:
grid_search.best_estimator_

custom_knn(distance_func=<function cosine at 0xa1c414d40>, n_neighbors=9)

Our best estimator then is a 9-nearest neighbors model using cosine distance.

In [100]:
print('Highest CV Mean Accuracy')
max(grid_search.cv_results_['mean_test_score'])

Highest CV Mean Accuracy


0.8146067415730337

Our highest CV mean accuracy is 81.4%, suggesting our classifier did quite well. We can also examine specificity/sensitivity as well to see if our model performs equally well among all labels. 

In [104]:
# code taken from https://nbviewer.jupyter.org/github/jakemdrew/StrandPy/blob/master/StrandClassifierV2%20GridSearchCV%20RDP%20CategoryScores.ipynb
from sklearn.metrics import classification_report
yhats = np.zeros(len(wine.target))

#Perform 10 fold cv, saving all predictions 
for test, train in cv.split(wine.data,wine.target):
    # Train and test on the one fold
    knn = custom_knn(n_neighbors=9, distance_func=cosine)
    knn.fit(wine.data[train],wine.target[train])
    yhats[test] = knn.predict(wine.data[test])
    
    
print(classification_report(y_true = wine.target, y_pred = yhats))


              precision    recall  f1-score   support

           0       0.85      0.86      0.86        59
           1       0.72      0.58      0.64        71
           2       0.46      0.58      0.51        48

    accuracy                           0.67       178
   macro avg       0.68      0.68      0.67       178
weighted avg       0.69      0.67      0.68       178



We see that our accuracy statistics were inflated by the strong performance in classifying 0's, but our model did a poor job in correctly classifying wine type 2. 